# Assistant-02<br/>Assistant Framework

This notebook showcases a foundational framework to work with Assistants.

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import common
import oaihelper as framework

## Get an OpenAI client
client = framework.get_openai_client()

## Create an assistant
math_tutor_assistant = framework.create_assistant(client,"Math Tutor","You are a personal math tutor. Write and run code to answer math questions.", [{"type": "code_interpreter"}], common.gpt_deployment_name)

## Test messages and recall the threads across users

In [2]:


new_message = framework.generate_response(client,math_tutor_assistant,"What's 2+2", "123", "John")

new_message = framework.generate_response(client,math_tutor_assistant,"What's x*x?", "456", "Sarah")

#new_message = framework.generate_response(client,math_tutor_assistant,"What the value of x, for x=m*y+5, if y=2 and m=-10?", "123", "John")

#new_message = framework.generate_response(client,math_tutor_assistant,"What is the value of y, for y=x*x?, if x=2?", "456", "Sarah")

Retrieving existing thread for John with wa_id 123
Generated message: 2 + 2 equals 4.
To John: 2 + 2 equals 4.
Retrieving existing thread for Sarah with wa_id 456
Generated message: The expression \( x \times x \) simplifies to \( x^2 \), which represents the square of the variable \( x \).
To Sarah: The expression \( x \times x \) simplifies to \( x^2 \), which represents the square of the variable \( x \).


In [3]:
golang_tutor = framework.create_assistant(client,"Golang","You are an assistant that can help write and explain Go code.", None, common.gpt_deployment_name)

In [4]:
new_message = framework.generate_response(client,golang_tutor,"Write a Go application to find the 1001st prime number.", "789", "Victor")

Retrieving existing thread for Victor with wa_id 789
Generated message: Below is a Go application that calculates the 1001st prime number. You can run it on your local development setup.

```go
package main

import (
	"fmt"
)

// isPrime checks if a number is a prime number by checking divisibility
// on numbers less than or equal to its square root.
func isPrime(number int) bool {
	if number <= 1 {
		return false
	}
	if number == 2 {
		return true
	}
	if number%2 == 0 {
		return false
	}
	for i := 3; i*i <= number; i += 2 {
		if number%i == 0 {
			return false
		}
	}
	return true
}

// findNthPrime finds the nth prime number by iterating over natural numbers
// and checking for prime status until n primes have been found.
func findNthPrime(n int) int {
	count := 0
	number := 1
	for count < n {
		number++
		if isPrime(number) {
			count++
		}
	}
	return number
}

func main() {
	nthPrimeNumber := findNthPrime(1001)
	fmt.Printf("The 1001st prime number is: %d\n", nthPrimeNumber)
}
```

S

In [5]:
#csv_analyzer_tutor = framework.create_assistant(client,"","You are an assistant that can help write and explain Go code.", [{"type": "code_interpreter"}], common.gpt_deployment_name)
web_parser_assistant = framework.create_assistant(client, "Web Parser", "You are an assistant that can get scrape a web page and download the content.", [{"type": "code_interpreter"}], common.gpt_deployment_name)

In [6]:
new_message = framework.generate_response(client,web_parser_assistant,"Can you help call the website at https://api.github.com/users/Welasco and get the content.", "789", "Victor")

Retrieving existing thread for Victor with wa_id 789
Generated message: It appears I made an error by attempting to perform an action that I am not capable of due to the restrictions of my current environment. I do not have the ability to directly access or interact with external websites, including the GitHub API.

To fetch the content from `https://api.github.com/users/Welasco`, you will need to execute the Python script provided in previous messages on your own local machine, which has access to the internet.

If you require assistance in any other way or have questions on how to execute the code, please feel free to ask!
To Victor: It appears I made an error by attempting to perform an action that I am not capable of due to the restrictions of my current environment. I do not have the ability to directly access or interact with external websites, including the GitHub API.

To fetch the content from `https://api.github.com/users/Welasco`, you will need to execute the Python script p

In [7]:
framework.cleanup(client)

AssistantDeleted(id='asst_m3VuNaQz8DQLMbxBmjrPFYnL', deleted=True, object='assistant.deleted')
AssistantDeleted(id='asst_5k5iTP8PXB6dXdTXn3sylo5r', deleted=True, object='assistant.deleted')
AssistantDeleted(id='asst_W9IC1zZYLGUMQfEtskJav9Dm', deleted=True, object='assistant.deleted')
